## Baseline
### accuracy: 0.9726872246696036
- Only Chinese Characters + jieba + remove stopwords
- Tokenizer
- TfidfVectorizer
- Ensemble model

### 改进方向
- 保留日期、英文缩写
- 存在英文content，需要处理
- lightgbm参数调优
- 融合 bert-like transformer

In [1]:
import pandas as pd
import json
import os
import re
import jieba
from tqdm import tqdm
from paddlenlp import Taskflow
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from transformers import PreTrainedTokenizerFast
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

C:\ProgramData\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
train_dir = './train.csv'
test_dir = './test.csv'
df1 = pd.read_csv(train_dir)
df2 = pd.read_csv(test_dir)

In [3]:
train = df1[['id', 'content', 'topic']]
test = df2[['id', 'content', 'topic']]
test

,id,content,topic
0,181,2023年，习近平主席亲自擘画引领，中国特色大国外交扎实推进。在推动构建人类命运共同体的崇高...,1
1,1763,当地时间11月16日，习近平主席向亚太经合组织（APEC）工商领导人峰会发表书面演讲，强调我...,1
2,1015,亚太经合组织第三十次领导人非正式会议在美国旧金山莫斯科尼中心举行。国家主席习近平出席会议并发...,1
3,7104,习近平出席亚太经合组织领导人同东道主嘉宾非正式对话会暨工作午宴央视网2023年11月17日 ...,1
4,2427,中国国家主席习近平将于11月14日应美国总统拜登邀请赴美举行中美领导人会晤，同时应邀出席亚太...,1
...,...,...,...
3400,1044,从头到尾来讲一遍，咱们按照惯例，先谈谈缅甸北部的那场风波。这场风波的源头是藏匿在缅甸北部的一...,23
3401,9043,缅北全线激战，敏昂莱誓言反击，冲突可能扩大化，中缅边境乱不得这几天，缅甸内战的爆发，引起了外...,23
3402,5570,近日，缅甸内战再次卷土重来，引起了国际社会广泛的关注。然而，这次的冲突异常纷乱，各种消息传来...,23
3403,5523,缅北战火越烧越大，中方做两手准备，高层直接去了缅甸首都清酒半栩换种角度分析故事，带你了解更有...,23


In [4]:
def get_stopword():
    stopwords = []
    with open('baidu_stopwords.txt', 'r', encoding='utf-8') as f:
        for line in f.readlines():
            stopwords.append(line.strip())
   
    stopwords = list(set(stopwords))
    return stopwords
stopwords = get_stopword()

In [6]:
def wordopt_cn(text):
    # only chinese characters
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese_txt = re.sub(pattern, '', text)
    return str(chinese_txt)

def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stopwords]
    return ' '.join(words)


train_copy = train.copy()
train_copy['content'] = train_copy['content'].apply(wordopt_cn)
train_copy['content'] = train_copy['content'].apply(lambda x: ' '.join(jieba.cut(x)))
train_copy['content'] = train_copy['content'].apply(remove_stopwords)
train_copy['content'] = train_copy['content'].apply(lambda x: re.sub('\\s+', ' ', x))
train_copy.drop(train_copy[train_copy['content'] == ''].index, inplace=True)
train = train_copy


Building prefix dict from the default dictionary ...
[2024-02-22 20:25:05,604] [   DEBUG] __init__.py:113 - Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shens\AppData\Local\Temp\jieba.cache
[2024-02-22 20:25:05,605] [   DEBUG] __init__.py:132 - Loading model from cache C:\Users\shens\AppData\Local\Temp\jieba.cache
Loading model cost 0.466 seconds.
[2024-02-22 20:25:06,071] [   DEBUG] __init__.py:164 - Loading model cost 0.466 seconds.
Prefix dict has been built successfully.
[2024-02-22 20:25:06,071] [   DEBUG] __init__.py:166 - Prefix dict has been built successfully.


In [7]:
train

,id,content,topic
0,108,当地 时间 月 日 上午 亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科...,1
1,7874,推动 中美关系 重回 正轨 领航 亚太 发展 繁荣 新程 国际 社会 期待 习近平 主席 赴...,1
2,7388,月 日 国务院新闻办公室 发布 携手 构建 人类 命运 共同体 中国 倡议 白皮书 新闻 发...,1
3,5271,亚太经合组织 第二十七次 领导人 非正式 会议 亚太经合组织 最高级别 会议 亚太经合组织 ...,1
4,3781,当地 时间 月 日 上午 亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科...,1
...,...,...,...
6805,2604,缅甸 再起 战火 地方 武装 袭击 多个 军事 据点 宣称 剿灭 电信 诈骗 缅甸 北部 燃...,23
6806,3157,中国 军队 采取行动 前往 中 缅 边境 缅甸 内战 发出信号 缅甸 内战 如今 越发 激烈...,23
6807,1567,正文 申请 入驻 缅甸 内战 争夺 来源 宋鸿兵 北京 举报 分享 至用 微信 扫码 二维码...,23
6808,9353,切换 模式 写文章 登录 注册 中国 缅甸 划清 边界 解放军 还 缅甸 一仗 沈听雪 喜欢...,23


In [8]:
def process_text(text):
    text = wordopt_cn(text)
    text = ' '.join(jieba.cut(text))
    text = remove_stopwords(text)
    text = re.sub('\\s+', ' ', text)
    return text

test['content'] = test['content'].apply(process_text)
test

C:\Users\shens\AppData\Local\Temp\ipykernel_78324\1294142915.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['content'] = test['content'].apply(process_text)


,id,content,topic
0,181,年 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人类 命...,1
1,1763,当地 时间 月 日 习近平 主席 亚太经合组织 工商 领导人 峰会 发表 书面 演讲 秉持 ...,1
2,1015,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1
3,7104,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 年...,1
4,2427,中国 国家 主席 习近平 月 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会晤 应邀...,1
...,...,...,...
3400,1044,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23
3401,9043,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23
3402,5570,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23
3403,5523,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23


In [9]:
LOWERCASE = False
VOCAB_SIZE = 32768-256

In [11]:
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

dataset = Dataset.from_pandas(test[['content']])


def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i: i + 1000]["content"]


raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['content'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['content'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

100%|██████████| 6803/6803 [00:09<00:00, 712.65it/s]


In [12]:
def dummy(text):
    return text


vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

vectorizer.fit(tokenized_texts_test)

vocab = vectorizer.vocabulary_

vectorizer = TfidfVectorizer(lowercase=False, sublinear_tf=True, vocabulary=vocab,
                             analyzer='word',
                             tokenizer=dummy,
                             preprocessor=dummy,
                             token_pattern=None, strip_accents='unicode'
                             )

X_train = vectorizer.fit_transform(tokenized_texts_train)


In [13]:
print(len(vocab))

30529


In [16]:
clf = MultinomialNB(alpha=0.02)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber")
p6 = {
    'n_iter': 1500, 
    'verbose': -1, 
    'objective': 'multiclass', 
    'metric': 'multi_logloss', 
    'learning_rate': 0.05073909898961407,
    'colsample_bytree': 0.726023996436955, 
    'colsample_bynode': 0.5803681307354022, 
    'lambda_l1': 8.562963348932286,
    'lambda_l2': 4.893256185259296, 
    'min_data_in_leaf': 115, 
    'max_depth': 23, 
    'max_bin': 898,
    'num_class': 24  
}
lgb = LGBMClassifier(**p6)

# Creating the ensemble model
ensemble = VotingClassifier(estimators=[
    ('mnb', clf),
    ('sgd', sgd_model),
    ('lgb', lgb)],
    weights=[0.1, 0.45, 0.45],
    voting='soft',
    n_jobs=-1)

In [17]:
Y_train = train['topic'].values
ensemble.fit(X_train, Y_train)

VotingClassifier(estimators=[('mnb', MultinomialNB(alpha=0.02)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=8000, tol=0.0001)),
                             ('lgb',
                              LGBMClassifier(colsample_bynode=0.5803681307354022,
                                             colsample_bytree=0.726023996436955,
                                             lambda_l1=8.562963348932286,
                                             lambda_l2=4.893256185259296,
                                             learning_rate=0.05073909898961407,
                                             max_bin=898, max_depth=23,
                                             metric='multi_logloss',
                                             min_data_in_leaf=115, n_iter=1500,
                                             num_class=24,
                                             objective='multiclass',
                                             verbose=-1))],
                 n_jobs=-1, voting='soft', weights=[0.1, 0.45, 0.45])

In [21]:
tf_test = vectorizer.transform(tokenized_texts_test)
pred = ensemble.predict(tf_test)
test["pred"] = pred

C:\Users\shens\AppData\Local\Temp\ipykernel_78324\3150750775.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred"] = pred


In [22]:
test

,id,content,topic,pred
0,181,年 习近平 主席 亲自 擘画 引领 中国 特色 大国 外交 扎实 推进 推动 构建 人类 命...,1,22
1,1763,当地 时间 月 日 习近平 主席 亚太经合组织 工商 领导人 峰会 发表 书面 演讲 秉持 ...,1,1
2,1015,亚太经合组织 第三十次 领导人 非正式 会议 美国 旧金山 莫斯科 尼 中心 国家 主席 习...,1,1
3,7104,习近平 出席 亚太经合组织 领导人 东道主 嘉宾 非正式 对话会 暨 工作 午宴 央视网 年...,1,1
4,2427,中国 国家 主席 习近平 月 日应 美国 总统 拜登 邀请 赴美 中 美 领导人 会晤 应邀...,1,1
...,...,...,...,...
3400,1044,从头到尾 来讲 一遍 惯例 先 谈谈 缅甸 北部 那场 风波 这场 风波 源头 藏匿在 缅甸...,23,23
3401,9043,缅北 全线 激战 敏昂 莱 誓言 反击 冲突 扩大化 中 缅 边境 乱 几天 缅甸 内战 爆...,23,23
3402,5570,近日 缅甸 内战 再次 卷土重来 国际 社会 关注 这次 冲突 异常 纷乱 消息 传来 人 ...,23,23
3403,5523,缅北 战火 越烧 越大 中方 做 两手 高层 去 缅甸 首都 清酒 半栩换种 角度 分析 故...,23,23


In [27]:
accuracy = accuracy_score(test['topic'], test['pred'])
precision = precision_score(test['topic'], test['pred'], average='weighted')
recall = recall_score(test['topic'], test['pred'], average='weighted')
f1 = f1_score(test['topic'], test['pred'], average='weighted')
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1: {f1}')

Accuracy: 0.9726872246696036
Precision: 0.9728530390174358
Recall: 0.9726872246696036
F1: 0.972659429983902
